# Combined Client Example

This notebook demonstrates how to use the combined `LangGateLocal` client, which provides access to both registry and parameter transformation functionality in a single client.

In [1]:
import os
from pprint import pprint as pp

# Optional environment variables
# os.environ["LANGGATE_CONFIG"] = "path/to/langgate_config.yaml"
# os.environ["LANGGATE_MODELS"] = "path/to/langgate_models.json"
# os.environ["LOG_LEVEL"] = "debug"

## Initialize the Combined Client

In [2]:
from langgate.sdk import LangGateLocal

# Initialize the combined client
# LangGateLocal is a singleton that combines both registry and transformer functionality
client = LangGateLocal()

## List Available Models

In [3]:
# List available LLMs
llms = await client.list_llms()
print(f"Available LLMs: {len(llms)}")
for model in llms[:5]:
    print(f"- {model.id}: {model.name}")

print("\n" + "=" * 50 + "\n")

# List available image models
image_models = await client.list_image_models()
print(f"Available Image Models: {len(image_models)}")
for model in image_models[:5]:
    print(f"- {model.id}: {model.name}")

2025-08-18 02:23:46 [debug    ] refreshing_model_caches       
2025-08-18 02:23:46 [debug    ] refreshed_model_caches         image_count=4 llm_count=5
Available LLMs: 5
- openai/gpt-5-chat: ChatGPT-5
- openai/gpt-5: GPT-5
- openai/gpt-5-high: GPT-5 high
- anthropic/claude-sonnet-4: Claude-4 Sonnet
- anthropic/claude-sonnet-4-reasoning: Claude-4 Sonnet R


Available Image Models: 4
- openai/gpt-image-1: GPT Image 1
- openai/dall-e-3: DALL-E 3
- black-forest-labs/flux-dev: FLUX.1 [dev]
- stability-ai/sd-3.5-large: SD 3.5 Large


## Define Input Parameters

In [4]:
# Sample user parameters
input_params = {
    "temperature": 0.7,
    "max_tokens": 20_000,
    "stream": True,
}

## Get Model Info and Transform Parameters

Using the combined client, we can get model information and transform parameters from a unified interface.

In [5]:
if llms:
    model_id = llms[0].id

    # Get model info
    model_info = await client.get_llm_info(model_id)
    print(f"Model: {model_info.name}")
    print(f"Provider: {model_info.provider.name}")
    print(f"Description: {model_info.description}")

    # Transform parameters
    transformed = await client.get_params(model_id, input_params)
    print("\nTransformed parameters:")
    pp(transformed)

Model: ChatGPT-5
Provider: OpenAI
Description: ChatGPT-5 from OpenAI is designed for advanced, natural, multimodal, and context-aware conversations.

Transformed parameters:
('openai',
 {'api_key': SecretStr('**********'),
  'base_url': 'https://api.openai.com/v1',
  'max_tokens': 20000,
  'model': 'gpt-5-chat-latest',
  'stream': True})


## Working with Model-Specific Configurations

Different models may have different parameter transformations defined in `langgate_config.yaml`.
You can define "virtual models" with IDs referring to a model + a configuration.

In [6]:
# Using a model with a specific configuration.
# Anthropic has no actual model with this ID. We define it ourselves,
# to reference claude-sonnet-4-0 with a specific reasoning configuration.
model_id = "anthropic/claude-sonnet-4-reasoning"

# Get model info
model_info = await client.get_llm_info(model_id)
print(f"Model: {model_info.name}")
print(f"Provider: {model_info.provider.name}")
print(f"Description: {model_info.description}")

# Transform parameters
transformed = await client.get_params(model_id, input_params)
print("\nTransformed parameters:")
pp(transformed)

Model: Claude-4 Sonnet R
Provider: Anthropic
Description: Claude-4 Sonnet with reasoning capabilities.

Transformed parameters:
('anthropic',
 {'api_key': SecretStr('**********'),
  'base_url': 'https://api.anthropic.com',
  'max_tokens': 20000,
  'model': 'claude-sonnet-4-0',
  'stream': True,
  'thinking': {'budget_tokens': 1024, 'type': 'enabled'}})


## Working with Model Capabilities

In [7]:
# Check model capabilities
if llms:
    model_id = llms[0].id
    model_info = await client.get_llm_info(model_id)

    print(f"Capabilities for {model_info.name}:")
    pp(model_info.capabilities.model_dump(exclude_none=True))

    # Adjust parameters based on capabilities
    custom_params = input_params.copy()

    # Example: Add tools if supported
    if model_info.capabilities.supports_tools:
        custom_params["tools"] = [
            {
                "type": "function",
                "function": {
                    "name": "get_weather",
                    "description": "Get the current weather in a location",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and country, e.g., Cork, Ireland",
                            }
                        },
                        "required": ["location"],
                    },
                },
            }
        ]

    # Transform the custom parameters
    transformed = await client.get_params(model_id, custom_params)
    print("\nTransformed parameters with tools:")
    pp(transformed)

Capabilities for ChatGPT-5:
{'supports_files': True,
 'supports_max_tokens': True,
 'supports_prompt_caching': True,
 'supports_reasoning': True,
 'supports_response_schema': True,
 'supports_seed': True,
 'supports_tools': False,
 'supports_vision': True}

Transformed parameters with tools:
('openai',
 {'api_key': SecretStr('**********'),
  'base_url': 'https://api.openai.com/v1',
  'max_tokens': 20000,
  'model': 'gpt-5-chat-latest',
  'stream': True})


## Working with Image Models

In [8]:
# Image model parameters
image_params = {
    "prompt": "A beautiful sunset over the ocean",
    "size": "1024x1024",
    "quality": "medium",
    "n": 1,
}

In [9]:
# Transform parameters for an image model - the same pattern as for LLMs
if image_models:
    image_model_id = image_models[0].id

    # Get image model info
    image_info = await client.get_image_model_info(image_model_id)
    print(f"Image Model: {image_info.name}")
    print(f"Provider: {image_info.provider.name}")
    print(f"Description: {image_info.description}")

    # Transform parameters
    transformed = await client.get_params(image_model_id, image_params)
    print("\nTransformed image generation parameters:")
    pp(transformed)

Image Model: GPT Image 1
Provider: OpenAI
Description: GPT-Image-1 is a multimodal model that accepts both text and image inputs, and produces image outputs.

Transformed image generation parameters:
('openai',
 {'api_key': SecretStr('**********'),
  'base_url': 'https://api.openai.com/v1',
  'model': 'gpt-image-1',
  'n': 1,
  'prompt': 'A beautiful sunset over the ocean',
  'quality': 'medium',
  'size': '1024x1024'})
